In [124]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [125]:
class Network(nn.Module):
    def __init__(self, input_size, hidden1_size, output_size, hidden2_size=None):
        super(Network, self).__init__()
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.output_size = output_size

        self.fc1 = nn.Linear(self.input_size, self.hidden1_size)
        self.fc2 = nn.Linear(self.hidden1_size, self.hidden2_size)
        self.fc3 = nn.Linear(self.hidden2_size, self.output_size)
    
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def predict(self, x):
        return self.forward(x)
    
    def train(self, X, y, epochs=1000, lr=0.01):
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            output = self.forward(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print(f'Epoch {epoch} loss: {loss.item()}')

In [126]:
df = pd.read_csv('car_data.csv')
df.head()

,Make,Price,Body Style,Mileage,MPG,Services
0,Chevrolet,"Price: $21,900",Sedan,"8,742 miles",33,3
1,Honda,"Price: $21,900",Sedan,"21,198 miles",35,9
2,Nissan,"Price: $20,000",Sedan,"60,388 miles",26,1
3,Nissan,"Price: $19,778",Sedan,"60,522 miles",26,8
4,Hyundai,"Price: $13,555",Sedan,"68,303 miles",34,6


In [127]:
for row in df.iterrows():
    price_str = row[1]['Price']
    character = 0
    for character in range(len(price_str)):
        if not price_str[0].isdigit():
            price_str = price_str.replace(price_str[0], '')
    if ',' in price_str:
        price_str = price_str.replace(',', '')
    df.at[row[0], 'Price'] = int(price_str)

    mileage_str = row[1]['Mileage']
    
    for character in mileage_str:
        if not character.isdigit():
            character = mileage_str.replace(character, '')
    if ' miles ' in mileage_str:
        mileage_str = mileage_str.replace('miles', '')
    if ',' in mileage_str:
        mileage_str = mileage_str.replace(',', '')
    df.at[row[0], 'Mileage'] = int(mileage_str)

In [128]:
from sklearn.preprocessing import LabelEncoder
body_style_encoder = LabelEncoder()
make_encoder = LabelEncoder()

df['Make'] = make_encoder.fit_transform(df['Make'])
df['Body Style'] = body_style_encoder.fit_transform(df['Body Style'])

In [129]:
df.head()

,Make,Price,Body Style,Mileage,MPG,Services
0,2,21900,0,8742,33,3
1,4,21900,0,21198,35,9
2,7,20000,0,60388,26,1
3,7,19778,0,60522,26,8
4,5,13555,0,68303,34,6


In [130]:
model = Network(6, 36, 1, 16)

In [131]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()

In [ ]:
df['Price'] = (df['Price'] - df['Price'].mean())/df['Price'].std()
df['Mileage'] = (df['Mileage'] - df['Mileage'].mean())/df['Mileage'].std()

df.head()

,Make,Price,Body Style,Mileage,MPG,Services
0,2,0.066859,0,-1.176607,33,3
1,4,0.066859,0,-0.818755,35,9
2,7,-0.10147,0,0.307146,26,1
3,7,-0.121138,0,0.310996,26,8
4,5,-0.67246,0,0.534538,34,6


In [134]:
array = df.to_numpy()
print(array)

[[2 21900 0 8742 33 3]
 [4 21900 0 21198 35 9]
 [7 20000 0 60388 26 1]
 [7 19778 0 60522 26 8]
 [5 13555 0 68303 34 6]
 [8 18995 0 96559 27 32]
 [7 21533 0 45263 26 3]
 [4 6995 0 85816 31 18]
 [7 20288 0 1695 38 2]
 [6 30000 0 8779 31 2]
 [7 18735 0 100250 26 3]
 [5 15000 0 48146 36 4]
 [6 18499 0 61131 33 2]
 [6 32444 0 17113 30 4]
 [3 12500 0 89636 24 30]
 [6 28890 0 15745 33 3]
 [6 13844 0 59244 35 14]
 [7 16816 0 32454 33 13]
 [1 64364 0 11819 32 5]
 [5 10529 0 109372 31 3]
 [8 23897 0 24500 37 4]
 [7 8318 0 105912 35 1]
 [0 27958 0 50712 29 8]
 [7 20750 0 9428 33 3]]


In [ ]:
X = torch.tensor(df.values, dtype=torch.float64)
Y = torch.tensor(df.values, dtype=torch.float64)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.